# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [2]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 1000)

#### Creating list of filepaths to process original event csv data files

In [3]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

c:\Users\danie\Projects\Udacity-Data Engineering Nanodegree\Data Modeling with Cassandra


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [4]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [50]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

NoHostAvailable: ('Unable to connect to any servers', {'127.0.0.1:9042': ConnectionRefusedError(10061, "Tried connecting to [('127.0.0.1', 9042)]. Last error: No connection could be made because the target machine actively refused it")})

#### Create Keyspace

In [52]:
session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

#### Set Keyspace

In [53]:
session.set_keyspace('udacity')

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




### For table artist_songs, session_id was used as a partition key because the queries will filter by this column. Column item_in_session was used as clustering column to help make a unique key.

In [54]:
session.execute('drop table if exists artist_songs')

session.execute("""
    CREATE TABLE IF NOT EXISTS artist_songs (
        artist text, 
        item_in_session int,
        length float,
        session_id int,
        song_title text,
        PRIMARY KEY(
            session_id, 
            item_in_session
        )
    )"""
)


In [55]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO artist_songs (artist, item_in_session, length, session_id, song_title)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (line[0], int(line[3]), float(line[5]), int(line[8]), line[9]))


#### Do a SELECT to verify that the data have been inserted into each table

In [56]:
def print_query(session, query):
    rows = session.execute(query)
    column_names = rows.column_names
    d = {col:[] for col in column_names}
    df = pd.DataFrame(d)

    for row in rows:
        list_values = [value for value in row]
        df.loc[len(df.index)] = list_values

    print(df)
    


In [57]:
print_query(session, "select artist, song_title, length from artist_songs where session_id=338 and item_in_session=4")

      artist                       song_title      length
0  Faithless  Music Matters (Mark Knight Dub)  495.307312


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

### For table user_songs, user_id and session_id were used as a partition key because the queries will filter by this column. Column item_in_session were used as clustering columns to help make a unique key.

In [58]:
session.execute('drop table if exists user_songs')

session.execute("""
    CREATE TABLE IF NOT EXISTS user_songs (
        user_id int,
        session_id int,
        item_in_session int,
        first_name text,
        last_name text,
        artist text,
        song_title text,
        PRIMARY KEY(
            (user_id,
            session_id),
            item_in_session
        )
    )"""
)

                    

In [60]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_songs (user_id, session_id, item_in_session, first_name, last_name, artist, song_title)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[1], line[4], line[0], line[9]))

In [61]:
print_query(session, "select artist, song_title, item_in_session, first_name, last_name from user_songs where user_id=10 and session_id=182")

              artist                                         song_title  item_in_session first_name last_name
0   Down To The Bone                                 Keep On Keepin' On              0.0     Sylvie      Cruz
1       Three Drives                                        Greece 2000              1.0     Sylvie      Cruz
2  Sebastien Tellier                                          Kilometer              2.0     Sylvie      Cruz
3      Lonnie Gordon  Catch You Baby (Steve Pitron & Max Sanna Radio...              3.0     Sylvie      Cruz


### For table artist_songs, song_title was used as a partition key because the queries will filter by this column. Column user_id was used as clustering column to help make a unique key.

In [62]:
session.execute('drop table if exists song_usage')
session.execute("""
    CREATE TABLE IF NOT EXISTS song_usage (
        user_id int,
        first_name text,
        last_name text,
        song_title text,
        PRIMARY KEY(
            song_title,
            user_id
        )
    )"""
)


In [63]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_usage (user_id, first_name, last_name, song_title)"
        query = query + " VALUES (%s, %s, %s, %s)"
        session.execute(query, (int(line[10]), line[1], line[4], line[9]))

In [64]:
print_query(session, "select user_id, first_name, last_name from song_usage where song_title='All Hands Against His Own'")

   user_id  first_name last_name
0     29.0  Jacqueline     Lynch
1     80.0       Tegan    Levine
2     95.0        Sara   Johnson


### Drop the tables before closing out the sessions

In [65]:
session.execute('drop table artist_songs')
session.execute('drop table user_songs')
session.execute('drop table song_usage')

### Close the session and cluster connection¶

In [66]:
session.shutdown()
cluster.shutdown()